## Import Libraries Needed

In [48]:
import collections
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

## Load Dataset

### Run on Google Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_data(input_file):
   
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [4]:
english_sentences = load_data('/content/drive/My Drive/Colab Notebooks/small_vocab_en.txt')
french_sentences = load_data('/content/drive/My Drive/Colab Notebooks/small_vocab_fr.txt')
print('Dataset Loaded')

Dataset Loaded


## Split Dataset into Train/Test Sets

## Examine Vocabulary

In [5]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Data Pre-processing Functions

In [6]:
# Tokenizing Function to transform
def tokenize(sentence):
    tokenizer = Tokenizer(char_level = False)
    tokenizer.fit_on_texts(sentence)
    return tokenizer.texts_to_sequences(sentence), tokenizer

In [7]:
# Padding Function
def pad(sentence, length=None):
    if length is None:
        length = max([len(sentence) for sentence in sentence])
    return pad_sequences(sentence, maxlen = length, padding = 'post')

In [8]:
# Preprocessing pipeline function
def preprocess(x, y):
    
    # Tokenize the input data
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    #Pad the Tokenized data
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

### Train Data Preprocessing

In [43]:
# Run Preprocessing pipeline on Data
processed_english_sentences, processed_french_sentences, eng_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

# Get train data lengths and sizes
max_english_sequence_length = processed_english_sentences.shape[1]
max_french_sequence_length = processed_french_sentences.shape[1]
english_vocab_size = len(eng_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Train Data Preprocessed')
print('Train data: English vocab size -', len(processed_english_sentences))
print('Train data: French vocab size -', len(processed_french_sentences))
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocab size:", english_vocab_size)
print("French vocab size:", french_vocab_size)

Train Data Preprocessed
Train data: English vocab size - 137861
Train data: French vocab size - 137861
Max English sentence length: 15
Max French sentence length: 21
English vocab size: 199
French vocab size: 344


## Modelling

In [60]:
# RNN model incorporating Bidirectional and Embedding techniques in Neural network

# Set Model Parameters and layers
model = Sequential()
# Embedding Layer
model.add(Embedding(input_dim=len(eng_tokenizer.word_index)+1,
                    output_dim=128,
                    input_length=processed_english_sentences.shape[1]))
# 1st Bidrectional layer - no feedback returned
model.add(Bidirectional(GRU(256,return_sequences=False)))
model.add(RepeatVector(processed_french_sentences.shape[1]))
# 2nd Embedding layer - feedback returned
model.add(Bidirectional(GRU(256,return_sequences=True)))
model.add(TimeDistributed(Dense(len(french_tokenizer.word_index)+1,activation='softmax')))
learning_rate = 0.005

# Compile Model parameters
model.compile(loss = sparse_categorical_crossentropy, 
              optimizer = Adam(learning_rate), 
              metrics = ['accuracy'])

In [133]:
# Fit Model on Data
model.fit(processed_english_sentences,processed_french_sentences, batch_size = 64, epochs = 100, validation_split = 0.2)

Epoch 1/100
1724/1724 [==============================] - 25s 14ms/step - loss: 0.5338 - accuracy: 0.8520 - val_loss: 0.6297 - val_accuracy: 0.8172
Epoch 2/100
1724/1724 [==============================] - 26s 15ms/step - loss: 0.5073 - accuracy: 0.8496 - val_loss: 0.5064 - val_accuracy: 0.8486
Epoch 3/100
1724/1724 [==============================] - 25s 15ms/step - loss: 0.6211 - accuracy: 0.8115 - val_loss: 0.6104 - val_accuracy: 0.8094
Epoch 4/100
1724/1724 [==============================] - 25s 15ms/step - loss: 0.6496 - accuracy: 0.7988 - val_loss: 0.8183 - val_accuracy: 0.7501
Epoch 5/100
1724/1724 [==============================] - 25s 15ms/step - loss: 0.8591 - accuracy: 0.7449 - val_loss: 0.7814 - val_accuracy: 0.7581
Epoch 6/100
1724/1724 [==============================] - 26s 15ms/step - loss: 0.8557 - accuracy: 0.7457 - val_loss: 0.8500 - val_accuracy: 0.7446
Epoch 7/100
1724/1724 [==============================] - 25s 15ms/step - loss: 0.8934 - accuracy: 0.7345 - val_loss: 1

In [55]:
# Save the fitted model
model.save('/content/drive/MyDrive/Colab Notebooks/Second_model_%')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Fitted_model_98%/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Fitted_model_98%/assets


In [146]:
# Load Fitted Model for use
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Fitted_model_98%')

In [155]:
def English_to_French(sentence):
    
    french_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
    french_id_to_word[0] = ''
    x = sentence
    # Get Id's for tokenized sentence
    sentence = [eng_tokenizer.word_index[word] for word in sentence.split()]
    # Pad Id's to uniform length
    sentence = pad_sequences([sentence], maxlen=processed_english_sentences.shape[-1], padding='post')
    # Predict french Id's using sentence Id's in Fitted Model
    predictions = model.predict(sentence, len(sentence))
  
    print(x, "in French:")
    print(' '.join([french_id_to_word[np.argmax(processed_english_sentences)] for processed_english_sentences in predictions[0]]))

In [161]:
phrase = 'he dislikes lemons grapes and mangoes'
English_to_French(phrase)

he dislikes lemons grapes and mangoes in French:
il déteste les citrons les raisins et les mangues            


In [1]:
import gc
collected = gc.collect()
print("Garbage collector: collected","%d objects." % collected) 

Garbage collector: collected 289 objects.
